<a href="https://colab.research.google.com/github/asdfasdf001234/2024-1-MLPRJ/blob/main/ResNet/resnet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ResNet101 구현 (Pre-trained)

In [1]:
# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

# dataset and transformation
from torchvision import datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from torchvision import models
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
import pandas as pd
from torchsummary import summary
import time
import copy
from tqdm import tqdm

#cutout함수
from imgaug import augmenters as iaa

In [2]:
from PIL import Image
from glob import glob
import random

### 데이터 로드

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_dir = '/content/drive/MyDrive/ResNet_Data'

In [5]:
def create_dataframe(data_path, label_list):
    df = pd.DataFrame(columns=["path", "label", "class_id"])
    img_list = glob(os.path.join(data_path, '*.jpg'))

    for img in img_list:
        file_name = os.path.splitext(os.path.basename(img))[0]
        label_index = int(file_name[0]) - 1  #진영님 코드와 동일
        if 0 <= label_index < len(label_list):
            label = label_list[label_index]
            new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
            df = pd.concat([df, new_data], ignore_index=True)

    df["path"] = df["path"].astype(str)
    df["label"] = df["label"].astype(str)
    df["class_id"] = df["class_id"].astype(int)

    return df

In [6]:
# 데이터프레임
train_path = data_dir + '/Training/Training/'
valid_path = data_dir + '/Validation/Validation/'
test_path = data_dir + '/Testing/Testing/'
label_list = ['spring', 'summer', 'fall', 'winter']

train_df = create_dataframe(train_path, label_list)
val_df = create_dataframe(valid_path, label_list)
test_df = create_dataframe(test_path, label_list)

In [7]:
#데이터 출력
print("Train DataFrame:")
print(train_df.head())
print("Validation DataFrame:")
print(val_df.head())
print("Test DataFrame:")
print(test_df.head())

Train DataFrame:
                                                path label  class_id
0  /content/drive/MyDrive/ResNet_Data/Training/Tr...  fall         2
1  /content/drive/MyDrive/ResNet_Data/Training/Tr...  fall         2
2  /content/drive/MyDrive/ResNet_Data/Training/Tr...  fall         2
3  /content/drive/MyDrive/ResNet_Data/Training/Tr...  fall         2
4  /content/drive/MyDrive/ResNet_Data/Training/Tr...  fall         2
Validation DataFrame:
                                                path label  class_id
0  /content/drive/MyDrive/ResNet_Data/Validation/...  fall         2
1  /content/drive/MyDrive/ResNet_Data/Validation/...  fall         2
2  /content/drive/MyDrive/ResNet_Data/Validation/...  fall         2
3  /content/drive/MyDrive/ResNet_Data/Validation/...  fall         2
4  /content/drive/MyDrive/ResNet_Data/Validation/...  fall         2
Test DataFrame:
                                                path label  class_id
0  /content/drive/MyDrive/ResNet_Data/Testing/Te

In [8]:
## 컷아웃 augmentation 추가
class IAAcutout:
    def __init__(self, nb_iterations, size):
        self.aug = iaa.Cutout(nb_iterations=nb_iterations, size=size)

    def __call__(self, img):
        img = np.array(img)
        img = self.aug(image=img)
        return Image.fromarray(img)


## 블러 augmentation 추가
class IAAGaussianBlur:
    def __init__(self, sigma):
        self.aug = iaa.GaussianBlur(sigma=sigma)

    def __call__(self, img):
        img = np.array(img)
        img = self.aug(image=img)
        return Image.fromarray(img)


In [9]:
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self , dataframe , transforms_):
        self.df = dataframe
        self.transforms_ = transforms_

    def __len__(self):
        return len(self.df)

    def __getitem__(self ,index):
        img_path = self.df.iloc[index]['path']
        img = Image.open(img_path).convert("RGB")
        transformed_img = self.transforms_(img)
        class_id = self.df.iloc[index]['class_id']
        return transformed_img , class_id

In [37]:
Transforms = v2.Compose([
    v2.Resize((224,224)), # image 크기 맞추기
    v2.RandomRotation(degrees=(-15,15)), #랜덤하게 회전
    v2.RandomHorizontalFlip(0.5), #좌우반전
    IAAcutout(nb_iterations=6, size=0.2), #컷아웃 추가
    #IAAGaussianBlur(sigma=0.6),
    v2.ToTensor(), # torch.tensor로 변환
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #dtype 변환
    ])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [38]:
BATCH_SIZE = 30 #10으로 했더니 acc가 너무 작아서 30으로 늘려보았습니다

#데이터 로더
train_dataset = BaseDataset(train_df , Transforms) # train_transforms
val_dataset = BaseDataset(val_df , Transforms)
test_dataset = BaseDataset(test_df , Transforms)
train_loader = DataLoader(train_dataset , batch_size=BATCH_SIZE , shuffle = True)
val_loader = DataLoader(val_dataset , batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset , batch_size=BATCH_SIZE)

In [39]:
ResNet101 = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)

In [40]:
print(ResNet101)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [41]:
def set_random_seed(seed_value):
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
seed_value = 42
set_random_seed(seed_value)

In [43]:
import torch.nn as nn
class MyResNet101(nn.Module):
    def __init__(self, pretrained_model):
        super(MyResNet101, self).__init__()
        self.backbone = pretrained_model

        #파라미터 고정(frozen)
        for param in self.backbone.parameters():
            param.requires_grad = False

        self.dropout = nn.Dropout(0.5)
        self.extra_layer = nn.Linear(1000, 4)



    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.extra_layer(x)
        x = x.squeeze(1)
        return x


In [44]:
myresnet101 = MyResNet101(ResNet101)

## 파인튜닝

In [70]:
EPOCHS = 10
logs = {"train_loss":[] , "train_acc":[] , "val_loss":[] , "val_acc":[]}

if os.path.exists('checkpoints') == False:
    os.mkdir('checkpoints')

criterion = nn.CrossEntropyLoss() #결과가 multi-variable이므로 crossentropy 사용
optimizer = optim.Adam(myresnet101.parameters(), lr=0.0001 )
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0.0000001)


patience = 15
counter = 0
best_loss = np.inf

In [71]:
def train(dataloader , model , loss_fn , optimizer , lr_scheduler):
    size = 0
    num_batches = len(dataloader)

    model.train()
    epoch_loss , epoch_correct = 0 , 0

    for i ,(data_ , target_) in enumerate(dataloader):
        data_, target_ = data_.to(device), target_.to(device)
        size += data_.size(0)
        # 모델 출력 계산
        output = model(data_)

        # 손실 계산
        loss = loss_fn(output, target_)
        epoch_loss += loss.item()

        # 예측 계산
        _, predict = torch.max(output, 1)

        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #에포크 정확도
        epoch_correct += ( predict == target_ ).type(torch.float).sum().item()


    train_acc = epoch_correct/size
    lr_scheduler.step()


    return train_acc , epoch_loss / num_batches


def test(dataloader , model , loss_fn):
    size = 0
    num_baches = len(dataloader)
    epoch_loss , epoch_correct= 0 ,0

    with torch.no_grad(): # grad 연산 X
        model.eval() # evaluation dropout 연산시
        for i, (data_ , target_) in enumerate(dataloader):
          data_, target_ = data_.to(device), target_.to(device)
          size += data_.size(0)

          # 모델 출력 계산
          output = model(data_)
          # 손실 계산
          loss = loss_fn(output, target_)
          epoch_loss += loss.item()

          # 예측 계산
          _, predict = torch.max(output, 1)
          #에포크 정확도
          epoch_correct += ( predict == target_ ).type(torch.float).sum().item()

    test_acc = epoch_correct/size
    return test_acc  , epoch_loss / num_baches

In [72]:
myresnet101.cuda()

MyResNet101(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [73]:
for epoch in tqdm(range(EPOCHS)):
    train_acc , train_loss = train(train_loader ,
                                   myresnet101 ,
                                   criterion ,
                                   optimizer ,
                                   lr_scheduler)

    val_acc , val_loss = test(val_loader , myresnet101 , criterion)
    print(f'epoch:{epoch} \
    train_loss = {train_loss:.4f} , train_acc:{train_acc:.4f} \
    val_loss = {val_loss:.4f} , val_acc:{val_acc:.4f} \
    learning rate: {optimizer.param_groups[0]["lr"]}')
    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)

 10%|█         | 1/10 [00:04<00:36,  4.03s/it]

epoch:0     train_loss = 1.3464 , train_acc:0.3681     val_loss = 1.3412 , val_acc:0.2714     learning rate: 9.755527298894293e-05


 20%|██        | 2/10 [00:08<00:32,  4.12s/it]

epoch:1     train_loss = 1.3509 , train_acc:0.3299     val_loss = 1.3490 , val_acc:0.3286     learning rate: 9.046039886902862e-05


 30%|███       | 3/10 [00:12<00:29,  4.19s/it]

...

epoch:7     train_loss = 1.3590 , train_acc:0.3368     val_loss = 1.3447 , val_acc:0.3286     learning rate: 9.639601130971383e-06


 90%|█████████ | 9/10 [00:37<00:04,  4.17s/it]

epoch:8     train_loss = 1.3769 , train_acc:0.3542     val_loss = 1.3614 , val_acc:0.2143     learning rate: 2.5447270110570813e-06


100%|██████████| 10/10 [00:41<00:00,  4.12s/it]

epoch:9     train_loss = 1.3454 , train_acc:0.3264     val_loss = 1.3540 , val_acc:0.3714     learning rate: 1e-07


In [74]:
test_acc , val_loss = test(test_loader , myresnet101 , criterion)
print(test_acc)

0.38461538461538464
